<a href="https://colab.research.google.com/github/balasubramaniyan-p/pdsnd_github/blob/master/Index_RSI__R_MACD_Bolinger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive

import os

drive.mount('/content/drive')

os.chdir(f"/content/drive/MyDrive/Mutual_funds/")

!pip install mftool==2.8 -q
# import module
from mftool import Mftool
obj = Mftool()
scheme_code_dict=obj.get_scheme_codes()

def calculate_rsi(data, window,close="Close",date="Date",format="%d-%m-%Y"):
    # Convert 'Date' column to datetime
    data['Date'] = pd.to_datetime(data[date],format=format)
    data=data.sort_values(by='Date')
    data[close] = data[close].astype(float)
    data = data.reset_index(drop=True)
    delta = data[close].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_williams_r(data, window, close="Close", date="Date", format="%d-%m-%Y"):
    # Check if the required columns are in the dataframe
    #if high not in data.columns or low not in data.columns or close not in data.columns or date not in data.columns:
    #    raise ValueError(f"Data must contain '{high}', '{low}', '{close}', and '{date}' columns")

    # Convert 'Date' column to datetime
    data[date] = pd.to_datetime(data[date], format=format)
    data = data.sort_values(by=date)

    # Ensure the relevant columns are float
    #data[high] = data[high].astype(float)
    #data[low] = data[low].astype(float)
    data[close] = data[close].astype(float)
    data = data.reset_index(drop=True)

    # Calculate the highest high and lowest low over the look-back window
    highest_high = data[close].rolling(window=window).max()
    lowest_low = data[close].rolling(window=window).min()

    # Calculate Williams %R
    williams_r = ((highest_high - data[close]) / (highest_high - lowest_low)) * -100

    # Return Williams %R as a new column in the original DataFrame
    #data['Williams %R'] = williams_r

    return williams_r




# Find keys by value
list_to_lookup= [
    'quant Flexi Cap Fund - Growth Option-Direct Plan',
    'ICICI Prudential Regular Savings Fund - Direct Plan - Growth',
    'Invesco India PSU Equity Fund - Direct Plan - Growth',
    'ICICI Prudential Income Optimizer Fund (FOF) - Direct Plan - Growth',
    'Axis ELSS Tax Saver Fund - Direct Plan - Growth Option',
    'ICICI Prudential Regular Gold Savings Fund (FOF) - Direct Plan -  Growth',
    'HDFC Equity Savings Fund - Growth Option - Direct Plan',
    'Axis Bluechip Fund - Direct Plan - Growth',
    'ICICI Prudential MNC Fund - Direct Plan - Growth Option',
    'Kotak Equity Savings Fund - Direct - Growth',
    'ICICI Prudential US Bluechip Equity Fund - Direct Plan -  Growth',
    'Axis Gold Fund - Direct Plan - Growth option',
    'Tata Balanced Advantage Fund-Direct Plan-Growth',
    'ICICI Prudential Asset Allocator Fund (FOF) - Direct Plan - Growth',
    'ICICI Prudential Balanced Advantage Fund - Direct Plan -  Growth',
    'ICICI Prudential Nifty Next 50 Index Fund - Direct Plan -  Growth',
    'ICICI Prudential Equity & Debt Fund - Direct Plan - Growth',
    'ICICI Prudential Exports & Services Fund - Direct Plan - Growth',
    'ICICI Prudential Nifty 50 Index Fund - Direct Plan Cumulative Option',
    'ICICI Prudential Focused Equity Fund - Direct Plan - Growth',
    'ICICI Prudential Value Discovery Fund - Direct Plan - Growth',
    'ICICI Prudential Commodities Fund - Direct Plan - Growth Option',
    'quant Multi Asset Fund-GROWTH OPTION-Direct Plan',
    'ICICI Prudential Dividend Yield Equity Fund Direct Plan Growth Option',
    'ICICI Prudential Multi-Asset Fund - Direct Plan - Growth',
    'Nippon India Large Cap Fund - Direct Plan Growth Plan - Growth Option',
    'ICICI Prudential Manufacturing Fund - Direct Plan - Cumulative Option',
    'ICICI Prudential Bluechip Fund - Direct Plan - Growth',
    'ICICI Prudential Smallcap Fund - Direct Plan - Growth',
    'Axis Small Cap Fund - Direct Plan - Growth',
    'ICICI Prudential MidCap Fund - Direct Plan -  Growth',
    'Axis Midcap Fund - Direct Plan - Growth',
    'ICICI PRUDENTIAL HOUSING OPPORTUNITIES FUND - Direct Plan Growth',
    'ICICI Prudential India Opportunities Fund - Direct Plan - Cumulative Option',
    'ICICI Prudential Nifty Alpha Low - Volatility 30 ETF FOF - Direct Plan - Growth',
    'Kotak Bluechip Fund - Growth - Direct',
    'quant Mid Cap Fund - Growth Option - Direct Plan',
    'ICICI Prudential Nifty Alpha Low - Volatility 30 ETF FOF - Direct Plan - IDCW',
    'ICICI Prudential Smallcap Fund - Direct Plan - IDCW',
    'ICICI Prudential Flexicap Fund - Direct Plan - Growth',
    'quant Focused Fund - Growth Option-Direct Plan',
    'ICICI Prudential Multicap Fund - Direct Plan -  Growth',
    'ICICI Prudential MidCap Fund - Direct Plan -  IDCW',
    'ICICI Prudential Infrastructure Fund - Direct Plan -  Growth',
    'quant Small Cap Fund - Growth Option - Direct Plan',
    'ICICI Prudential Nifty Next 50 Index Fund - Direct Plan -  IDCW',
    'Nippon India Small Cap Fund - Direct Plan Growth Plan - Growth Option',
    'Nippon India Growth Fund - Direct Plan Growth Plan - Growth Option',
    'Motilal Oswal Midcap Fund-Direct Plan-Growth Option'
]

tickers = [key for key, value in scheme_code_dict.items() if value in list_to_lookup]
# print(tickers)

# Download historical data for NESI
import pandas as pd
results=[]
for ticker in tickers:
  try:
        data_pre = obj.get_scheme_historical_nav(code=ticker)
        data=pd.DataFrame(data_pre['data'])
        data['RSI'] = calculate_rsi(data, 14,date="date",close="nav",format="%d-%m-%Y")
        data['williams_r'] = calculate_williams_r(data, 14,date="date",close="nav",format="%d-%m-%Y")
        # Get the latest RSI value
        latest_rsi = data['RSI'].iloc[-1]
        latest_williams_r=data['williams_r'].iloc[-1]
        # print(f"\nThe latest RSI(14) value for {data_pre['scheme_name']} is {latest_rsi}.")
        results.append([ticker,data_pre['scheme_name'],latest_rsi,latest_williams_r])
  except Exception as e:
    print(f"Error occurred while fetching data for {ticker}: {e}")

Results=pd.DataFrame(results,columns=['Scheme code','Scheme','RSI',"Williams R"]).sort_values(by='RSI',ascending=True).reset_index(drop=True)
Results.to_csv("./RSI.csv", index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
import os
import pandas as pd

drive.mount('/content/drive')

os.chdir(f"/content/drive/MyDrive/Mutual_funds/")

pd.read_csv("RSI.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Scheme code,Scheme,RSI,Williams R
0,120821,quant Multi Asset Fund-GROWTH OPTION-Direct Plan,27.714532,-100.000000
1,120828,quant Small Cap Fund - Growth Option - Direct ...,31.685357,-100.000000
2,120591,ICICI Prudential Smallcap Fund - Direct Plan -...,33.385093,-100.000000
3,120870,ICICI Prudential Smallcap Fund - Direct Plan -...,33.469388,-100.000000
4,120834,quant Focused Fund - Growth Option-Direct Plan,33.749378,-100.000000
5,120843,quant Flexi Cap Fund - Growth Option-Direct Plan,34.919579,-100.000000
6,120599,ICICI Prudential Multicap Fund - Direct Plan -...,35.827942,-100.000000
7,120505,Axis Midcap Fund - Direct Plan - Growth,36.578708,-100.000000
8,120841,quant Mid Cap Fund - Growth Option - Direct Plan,36.623416,-100.000000
9,118778,Nippon India Small Cap Fund - Direct Plan Grow...,37.099026,-100.000000


In [ ]:


ticker='120828'
!pip install mftool==2.8 -q
# import module
from mftool import Mftool
obj = Mftool()
data_pre = obj.get_scheme_historical_nav(code=ticker)
data=pd.DataFrame(data_pre['data'])
data['RSI'] = calculate_rsi(data, 14,date="date",close="nav",format="%d-%m-%Y")
(data['RSI'].iloc[-14:][::-1])

,RSI
2891,31.685357
2890,37.876753
2889,58.423278
2888,49.786592
2887,62.231837
2886,55.832689
2885,49.008060
2884,53.181989
2883,54.744132
2882,52.042328
